In [ ]:
!unzip /content/drive/MyDrive/TestImages.zip

In [ ]:
def __get_list_from_dict(self, data: dict, size_img):
        result = []
        keys = data.keys()
        dx = size_img[0]/self.image_size[0]
        dy = size_img[1]/self.image_size[1]
        
        result.append(int(data['ul'][0]/dy))
        result.append(int(data['ul'][1]/dx))

        result.append(int(data['br'][0]/dy))
        result.append(int(data['br'][1]/dx)) 
        return result    

In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import itertools, math
import pandas
from google.colab.patches import cv2_imshow
from tensorflow import keras 
import os
from glob import glob
import json


In [ ]:
# подготовка данных 
path_dataset = "/content/TestImages"
path_dataset_list = os.listdir(path_dataset) 

path_images = sorted([ os.path.join(path_dataset,path_image) for path_image in path_dataset_list if path_image.split('.')[-1] == 'JPG'], key = lambda a:a[:-5])
path_json =   sorted([os.path.join(path_dataset,path_json) for path_json in path_dataset_list if path_json.split('.')[-1] == 'json'], key = lambda a:a[:-5] )

In [ ]:
# удаление потерянных файлов
path_json_ = [i[:-5] for i in path_json]
del_ = []
for i in range(len(path_json)):
    if not path_images[i] in path_json_:
        del_.append(path_images[i])

for path in del_:
    path_images.remove(path)

In [ ]:
print(len(path_json))
print(len(path_images))

In [ ]:
path_images = sorted(path_images)
path_json = sorted(path_json)

In [ ]:
# разделение на тренировочную и обучающую выборку 
count_data = len(path_images)
per = .75
train_path_images = path_images[:int(per*count_data)]
train_path_json = path_json[:int(per*count_data)]

valid_path_images = path_images[int(per*count_data):]
valid_path_json = path_json[int(per*count_data):]

In [ ]:
for i in range(len(train_path_images)):
    print(train_path_images[i], train_path_json[i])

In [ ]:
print(count_data)
print(len(train_path_images))
print(len(valid_path_images))

In [ ]:
img = x = cv2.imread(train_path_images[0])
with open (train_path_json[0], 'r') as file:
        data = json.load(file)
print(data)
cv2.circle(img, data['ul'],10,(255,0,0),-1)
cv2.circle(img, data['ur'],10,(255,0,0),-1)
cv2.circle(img, data['br'],10,(255,0,0),-1)
cv2.circle(img, data['bl'],10,(255,0,0),-1)

cv2_imshow(img)

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
     #  генератор данных 
    def __init__(self, images_path_list, coords_path_list, batch_size, image_size = (512,512)): 
 
        self.image_size = image_size


        self.images_path_list = sorted(images_path_list)
        self.coords_path_list = sorted(coords_path_list)
        
        self.batch_size = batch_size
        self.on_epoch_end()

    def __len__(self):
        # кол-во пакетов в эпоху 
        return  int(np.ceil(len(self.images_path_list)/ self.batch_size))

    def on_epoch_end(self):
        # обновление индексов 
        self.indexes = np.arange(len(self.images_path_list))
        # np.random.shuffle(self.indexes)

    def read_image(self,path): 
        x = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        x = cv2.resize(x, (512,512), cv2.INTER_CUBIC)
        return x 
    
    def get_point_in_json(self, path_json):
        point = []
        img = cv2.imread(path_json[:-5],cv2.IMREAD_GRAYSCALE)
        h,w = img.shape
        with open (path_json, 'r') as file:
            data = json.load(file)
            point = self.__get_list_from_dict(data, (h,w))
        return point

    
    def __getitem__(self, index): 
        indexes = self.indexes[index *self.batch_size : (index+1)* self.batch_size]

        image = tf.convert_to_tensor([self.read_image(self.images_path_list[k])/255.0 for k in indexes], dtype= tf.float64)
        coords = np.array([self.get_point_in_json(self.coords_path_list[k]) for k in indexes])

        return image, coords

    def __get_list_from_dict(self, data: dict, size_img):
        result = []
        keys = data.keys()
        dx = size_img[0]/self.image_size[0]
        dy = size_img[1]/self.image_size[1]
        for key in keys:
            result.append(int(data[key][0]/dy))
            result.append(int(data[key][1]/dx))
            
        return result                                                                                                                                                                                                           

In [ ]:
batch = 8

In [ ]:
train_generator = DataGenerator(train_path_images, train_path_json, batch_size=batch)
valid_generator = DataGenerator(train_path_images, train_path_json, batch_size=batch)

In [ ]:
for line in train_generator:
    for i in range(8):
        img = line[0][i].numpy().copy()
        cv2.circle(img, (line[1][i][0],line[1][i][1]),10,(255,0,0),-1)
        cv2.circle(img, (line[1][i][2],line[1][i][3]),10,(255,0,0),-1)
        cv2.circle(img, (line[1][i][4],line[1][i][5]),10,(255,0,0),-1)
        cv2.circle(img, (line[1][i][6],line[1][i][7]),10,(255,0,0),-1)
        cv2_imshow(img*255)
        print(line[1][i])
    break

In [ ]:
vgg.trainable = False
# flatten the max-pooling output of VGG
flatten = vgg.output
flatten = Flatten()(flatten)
# construct a fully-connected layer header to output the predicted
# bounding box coordinates
bboxHead = Dense(256, activation="relu")(flatten)
bboxHead = Dense(128, activation="relu")(bboxHead)
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(4, activation="sigmoid")(bboxHead)
# construct the model we will fine-tune for bounding box regression
model = Model(inputs=vgg.input, outputs=bboxHead)


In [ ]:
vgg = VGG19(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(512, 512,3)))

In [ ]:
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard

In [ ]:
# MODEL

# INPUT 
inputs = tf.keras.layers.Input(shape=(512,512,1))

conv_1 = tf.keras.layers.Conv2D(name = 'conv_1', filters = 16, kernel_size = 3, strides = 2, padding = 'same', activation = tf.keras.activations.relu)(inputs)
conv_1 = tf.keras.layers.BatchNormalization()(conv_1) 

conv_2 = tf.keras.layers.Conv2D(name = 'conv_2', filters = 32, kernel_size = 3, strides = 2, padding = 'same', activation = tf.keras.activations.relu)(conv_1)
conv_2 = tf.keras.layers.BatchNormalization()(conv_2) 

conv_3 = tf.keras.layers.Conv2D(name = 'conv_3', filters = 128, kernel_size = 3, strides = 2, padding = 'same', activation = tf.keras.activations.relu)(conv_2)
conv_3 = tf.keras.layers.BatchNormalization()(conv_3) 

conv_4 = tf.keras.layers.Conv2D(name = 'conv_4', filters = 256, kernel_size = 3, strides = 2, padding = 'same', activation = tf.keras.activations.relu)(conv_3)
conv_4 = tf.keras.layers.BatchNormalization()(conv_4) 

conv_5 = tf.keras.layers.Conv2D(name = 'conv_5', filters = 256, kernel_size = 3, strides = 2, padding = 'same', activation = tf.keras.activations.relu)(conv_4)
conv_5 = tf.keras.layers.BatchNormalization()(conv_5) 

flat = tf.keras.layers.Flatten()(conv_5)




hidden_1 = tf.keras.layers.Dense(4096, activation='sigmoid')(flat)
hidden_1 = tf.keras.layers.BatchNormalization()(hidden_1)
hidden_2 = tf.keras.layers.Dense(4096, activation='sigmoid')(hidden_1)
hidden_2 = tf.keras.layers.BatchNormalization()(hidden_2)



output = tf.keras.layers.Dense(8, activation='relu')(hidden_2)

In [ ]:
model = tf.keras.Model(inputs = inputs, outputs = output)

In [ ]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.models import Model

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = 'adam' , loss=tf.keras.losses.CategoricalCrossentropy(), metrics = ['accuracy', 'mse', 'mean_absolute_error'])

In [ ]:
callbacks = [
          ModelCheckpoint("files/model.h5"),
          ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4),
          CSVLogger("files/data.csv"),
          TensorBoard(),
          EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=False)
     ]

In [ ]:
model.fit(train_generator, validation_data= valid_generator,
        epochs=250,
        callbacks=callbacks, )